In [1]:
import pandas as pd
import pickle
import re 
from dateutil import parser
from datetime import datetime
from collections import namedtuple
import matplotlib.pyplot as plt
import numpy as np

In [2]:
with open("coalition", 'rb') as f:
    coalition  = pickle.load(f)


# # party  Alliance during UPA in center
# nda_2009 = coalition['nda_2009']
# upa_2009 = coalition['upa_2009']

# # party  Alliance during NDA in center
# nda_2014 = coalition['nda_2014']  
# upa_2014 = coalition['upa_2014'] 


# nda_2019 = coalition['nda_2019']
# upa_2019 = coalition['upa_2019']

In [3]:
df = pd.read_excel('Election Date.xlsx')
map = {'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05', 'Jun': '06', 'Jul': '07', 'Aug': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'}

In [4]:
# df

In [5]:
dateRange = ['2009-2014', '2014-2019', '2019-2021']
state_party_info = {}
now = datetime.now()
centerElections = ['2009-05-22', '2014-05-16', '2019-05-23', now.strftime('%Y-%m-%d')]
centerParties = ['INC', 'BJP', 'BJP']
totalElections = len(centerParties)
for r in dateRange:
    if r not in state_party_info:
        state_party_info[r] = {}
    for i, date in enumerate(df[r]):
        state = df['State'][i]
        # if(state != 'arunachal pradesh'): continue
        if state not in state_party_info[r]:
            state_party_info[r][state] = {}
        if(pd.isna(date)): continue
        # print("part_date : ",date)
        x = re.split('\[(.*?)\]', date)
        # print(x)
        i = 0
        p, d = [], []
        while(i+1 < len(x)):
            party = x[i].replace(', ', '').strip()
            # print("party: ",party)
            if party not in state_party_info[r][state]:
                state_party_info[r][state][party] = []
            for da in x[i+1].strip().split(','):
                if da == '': 
                    state_party_info[r][state][party].append(now.strftime('%Y-%m-%d'))
                    continue
                try:
                    date = (datetime.strptime(da.strip(), "%d-%b-%y").strftime('%Y-%m-%d'))
                    # state_party_info[r][state][party].append(date)
                except:
                    try:
                        date = (datetime.strptime(da.strip(), "%d-%b-%Y").strftime('%Y-%m-%d'))
                        # state_party_info[r][state][party].append(date)
                    except:
                        try:
                            # there was an error here related to CPI(M) because of split over () above line 4
                            date = (datetime.strptime(da.strip(), "%d/%m/%Y").strftime('%Y-%m-%d'))
                        except:
                            print(da.strip())
                state_party_info[r][state][party].append(date)
            i+=2
            while(i < len(x) and x[i].strip() == ''): i+=1

In [11]:
def months_overlap(s1,e1, s2,e2):

    Range = namedtuple('Range', ['start', 'end'])

    r1 = Range(start=datetime.strptime(s1,'%Y-%m-%d'), end=datetime.strptime(e1,'%Y-%m-%d'))
    r2 = Range(start=datetime.strptime(s2,'%Y-%m-%d'), end=datetime.strptime(e2,'%Y-%m-%d'))
    latest_start = max(r1.start, r2.start)
    earliest_end = min(r1.end, r2.end)
    print(str(latest_start).split(" "))
    print(str(earliest_end).split(" "))
    delta = (earliest_end - latest_start).days + 1
    overlap = max(0, delta)
    return overlap // 30

In [12]:
months_overlap('2009-05-15', '2014-05-15', '2011-05-15', '2017-05-15')

['2011-05-15', '00:00:00']
['2014-05-15', '00:00:00']


36

In [27]:
# UPA at center 
# center = "UPA"
# startDate = "2009-05-22"
# endDate = "2014-05-16"
# data = state_party_info["2009-2014"]
# nda_alliance = nda_2009
# upa_alliance = upa_2009

# # NDA1
# center = "NDA1"
# startDate = "2014-05-26"
# endDate = "2019-05-30"
# data = state_party_info["2014-2019"]
# nda_alliance = nda_2014
# upa_alliance = upa_2014


# #NDA2
# center = "NDA2"
# startDate = "2019-05-30"
# endDate = "2021-06-11"
# data = state_party_info["2019-2021"]
# nda_alliance = nda_2019
# upa_alliance = upa_2019


# center = "NDA"

# 
# state_party_info

In [30]:
state_party_info
new_state_party_date_info = {}

for timeline in  state_party_info:
    new_state_party_date_info[timeline] = {}
    ls_election_year = timeline.split('-')[0]
    nda_coalition  = coalition['nda_' + ls_election_year]
    upa_coalition  = coalition['upa_' + ls_election_year]
    
    for states in state_party_info[timeline]:
#         print(states)
        new_state_party_date_info[timeline][states] = {}
        for party in state_party_info[timeline][states]:
#             print(party)
            if party.lower() in nda_coalition and party.lower() != 'sp':
                new_state_party_date_info[timeline][states]["NDA"] = state_party_info[timeline][states][party]
            elif party.lower() in upa_coalition:
                new_state_party_date_info[timeline][states]["UPA"] = state_party_info[timeline][states][party]
            else:
                new_state_party_date_info[timeline][states]["TF"] = state_party_info[timeline][states][party]
        


In [29]:
# new_state_party_date_info

# with open('state_caolition_date', 'wb') as f:
#     pickle.dump(new_state_party_date_info, f)

In [ ]:
state_list, nda_list, upa_list, na_list = [],[],[],[]
for state in data:
    NDA, UPA, Non_Aligned =  0,0,0
    electrol_info =data[state]
    
    if electrol_info != {}:
#         print(electrol_info)
        for party in electrol_info:
            if party.lower() in upa_alliance:
                UPA += months_overlap(startDate,endDate, electrol_info[party][0], electrol_info[party][1] )
            elif party.lower() in nda_alliance and party.lower() != 'sp':
                NDA += months_overlap(startDate,endDate, electrol_info[party][0], electrol_info[party][1] )

            else:
                Non_Aligned += months_overlap(startDate,endDate, electrol_info[party][0], electrol_info[party][1] )
        state_list.append(state)
        nda_list.append(NDA)
        upa_list.append(UPA)
        na_list.append(Non_Aligned)
#         print(state, NDA, UPA, Non_Aligned )

# print(state_list)
# print(nda_list)
# print(upa_list)
# print(na_list)

coalition_months_NDA1 = np.stack([
                             np.array(nda_list),\
                             np.array(upa_list), \
                             np.array(na_list)
                            ]).T


# creating a list of index names
index_values = np.array(state_list)
   
# creating a list of column names
column_values = ['NDA', 'UPA', 'ThirdFront']
  
# creating the dataframe
coalition_months_NDA1 = pd.DataFrame(data = coalition_months_NDA1, 
                  index = index_values, 
                  columns = column_values)


with open("coalition_months_NDA1", 'wb') as f:
    pickle.dump(coalition_months_NDA1, f)


In [ ]:
if center == "NDA":
    with open('./coalition_months_NDA', 'rb') as f:
        coalition_months_NDA = pickle.load(f)
    coal = coalition_months_NDA
    state_list = coal.index.values.tolist()
    nda_list = list(coal['NDA'])
    upa_list = list(coal['UPA'])
    na_list =list(coal['ThirdFront'])


In [ ]:


fig, ax = plt.subplots( figsize =(12, 8))
fig.suptitle("Center State Alliance: {} center govt".format(center))

width = 0.6

br = np.arange(len(state_list))

ax.bar(br, nda_list, width, label='NDA', color='#F97306')
ax.bar(br, np.array(upa_list), width, bottom=np.array(nda_list),
       label='UPA', color='b')

ax.bar(br, na_list, width, bottom=np.sum([np.array(nda_list),np.array(upa_list)],axis=0),
       label='ThirdFront', color='g')

ax.set_xticks([r for r in range(len(state_list))]) 
ax.set_xticklabels(state_list, fontsize=12,  rotation = 90)

ax.set_ylabel('Months')
ax.set_title('State level alliance with {} as center govt'.format(center))
ax.legend()
plt.tight_layout()
plt.show()
fig.savefig("{}".format(center))